In [2]:
from __future__ import print_function, absolute_import, division

import numpy as np
import pandas as pd
pd.set_option("max_columns", None)
import matplotlib
from matplotlib import pyplot as plt
#import seaborn as sns
%precision 2

import os
import ast
import csv
import time
import datetime
import json
import random
import math
from time import ctime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

import gym

from collections import namedtuple, deque

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

is_ipython = "inline" in matplotlib.get_backend()
if is_ipython:
    from IPython import display

'%.2f'

In [52]:
class DQN:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_min = 0.05
        self.epsilon_decay = (self.epsilon - self.epsilon_min) / 50000
        self.batch_size = 64
        self.memory = deque(maxlen=2000)
        self.episodes = 1000
        self.train_start = 1000
        self.model = self.build_model()
        self.target_model = self.build_model()
        self.update_target_model()
    
    def build_model(self):
        model = Sequential()
        model.add(Dense(32, input_dim=self.state_size, activation="relu", kernel_initializer="he_uniform"))
        model.add(Dense(16, activation="relu", kernel_initializer="he_uniform"))
        model.add(Dense(self.action_size, activation="linear", kernel_initializer="he_uniform"))
        model.compile(loss="mse",optimizer=Adam(lr=self.learning_rate))
        model.summary()
        return model
    
    def act(self, state):
        if self.epsilon<np.random.rand():
            return np.random.choice(np.arange(self.action_size))
        else:
            act_values = self.model.predict(state)
            return np.argmax(act_values[0])
    
    def remember(self,state, action, reward, next_state, done):
        if action == 2:
            action = 1
        self.memory.append((state, action, reward, next_state, done))
        if self.epsilon > self.epsilon_min:
            self.epsilon -= self.epsilon_decay
    
    def replay(self):
        if len(self.memory)<self.train_start:
            return
        batch_size = min(len(self.memory),self.batch_size)
        mini_batch = random.sample(self.memory, batch_size)
        update_input = np.zeros((batch_size, self.state_size))
        update_target = np.zeros((batch_size, self.state_size))
        for i in range(batch_size):
            state, action, reward, next_state, done = mini_batch[i]
            target = self.model.predict(state)[0]
            if done:
                target[action] = reward
            else:
                target[action] = reward + self.gamma* np.amax(self.target_model.predict(next_state)[0])
            update_input[i] = state
            update_target[i] = target
        self.model.fit(update_input, update_target, batch_size=batch_size, epochs=1, verbose=0)
    
    def load_model(self, name):
        self.model.load_weights(name)
    
    def save_model(self, name):
        self.model.save_weights(name)
    
    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

In [ ]:
if __name__ == "__main__":
    env = gym.make('MountainCar-v0')
    state_size = env.observation_space.shape[0]
    action_size = 2
    agent = DQN(state_size, action_size)
    #agent.load_model("./save_model/MountainCar_DQN.h5")
    scores, episodes = [], []

    for e in range(agent.episodes):
        done = False
        score = 0
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        print(state)
        fake_action = 0
        action_count = 0
        while not done:
#             if agent.render:
#                 env.render()
            action_count = action_count + 1
            if action_count == 4:
                action = agent.act(state)
                action_count = 0
                if action == 0:
                    fake_action = 0
                elif action == 1:
                    fake_action = 2
            #print("fake_action ",fake_action, "action ", action)
            next_state, reward, done, info = env.step(fake_action)
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, fake_action, reward, next_state, done)
            agent.replay()
            score += reward
            state = next_state

            if done:
                env.reset()
                agent.update_target_model()
                scores.append(score)
                episodes.append(e)
                print("episode:", e, "  score:", score, "  memory length:", len(agent.memory),
                      "  epsilon:", agent.epsilon)
#         if e % 50 == 0:
#              agent.save_model("./save_model/MountainCar_DQN.h5")


I0506 22:19:35.848170 140423791109952 registration.py:117] Making new env: MountainCar-v0
[2019-05-06 22:19:35,848] Making new env: MountainCar-v0


Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 32)                96        
_________________________________________________________________
dense_49 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_50 (Dense)             (None, 2)                 34        
Total params: 658
Trainable params: 658
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_51 (Dense)             (None, 32)                96        
_________________________________________________________________
dense_52 (Dense)             (None, 16)                528       
________________________________

array([-0.5,  0. ])

episode: 0   score: -200.0   memory length: 200   epsilon: 0.9962000000000018
[[-0.6  0. ]]


array([-0.55,  0.  ])

...

episode: 73   score: -200.0   memory length: 2000   epsilon: 0.7195790000001292
[[-0.56  0.  ]]


array([-0.46,  0.  ])

episode: 74   score: -200.0   memory length: 2000   epsilon: 0.715779000000131
[[-0.48  0.  ]]


array([-0.55,  0.  ])

episode: 75   score: -200.0   memory length: 2000   epsilon: 0.7119790000001327
[[-0.43  0.  ]]
